In [ ]:
!python --version

In [59]:
import pandas as pd

In [60]:
import yaml

In [61]:
import time

In [62]:
from datetime import datetime

In [63]:
print(datetime.now())

2022-03-08 16:28:03.940280


In [64]:
f = open("Milestone2A_Log.txt", "w")

In [65]:
#opening in yaml
data =""
with open('Milestone2\MileStone2A.yaml') as file:
    content = yaml.safe_load_all(file)
    data=next(content)
print(datetime.now())
f.write(f"{datetime.now()};M2A_Workflow Entry \n")
main_flow = data['M2A_Workflow']

2022-03-08 16:28:04.176843


In [66]:
#execting time function
def TimeFunction(inputs,path,task,condition):
    lock.acquire()
    print(condition)
    #cond = ""
    if(len(condition)>0):
        s=condition[2:len(condition)-5]
        print(s)
        print(returnDict[s])
        
        num = eval(condition[len(condition)-1])
        if(condition[len(condition)-3] == '>'):
            
            if returnDict[s]>num:
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return
        elif condition[len(condition)-3] == '<':
            if(returnDict[s]<num):
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return
        #cond = eval(s1)
        #print(cond)
        #cond = eval(s1)
        #if cond:
         #   print("true")
        #else:
         #   print("false")
    f.write(f"{datetime.now()};{path} Executing TimeFunction ({inputs['FunctionInput']},{inputs['ExecutionTime']}) \n")
    #print(execution_time)
    print("executing ......")
    time.sleep(int(inputs['ExecutionTime']))
    print("done...")
    if(len(task) == 0):
        f.write(f"{datetime.now()};{path} Exit \n")
    else:
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    lock.release()

In [67]:
def DataLoad(inputs,path,task,condition):
    lock.acquire()
    print(condition)
    #cond=""
    if(len(condition)>0):
        s=condition[2:len(condition)-5]
        #cond = eval(s1)
        print(s)
        print(returnDict[s])
        num = eval(condition[len(condition)-1])
        if(condition[len(condition)-3] == '>'):
            
            if returnDict[s]>num:
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return 
        elif condition[len(condition)-3] == '<':
            if(returnDict[s]<num):
                print("true")
            else:
                print(path+" skipped")
                f.write(f"{datetime.now()};{path} Skipped \n")
                f.write(f"{datetime.now()};{path} Exit \n")
                print("false")
                lock.release()
                return 
        #cond = eval(s1)
        #print(cond)
        #if cond:
          #  print("true")
        #else:
           # print("false")
    print(inputs['Filename'])
    f.write(f"{datetime.now()};{path} Executing DataLoad ({inputs['Filename']}) \n")
    df = pd.read_csv(f"Milestone2\{inputs['Filename']}")
    if(len(task) == 0):
        f.write(f"{datetime.now()};{path} Exit \n")
    else:
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    print("done.....")
    lock.release()
    
    return df,len(df)
    

In [68]:
import threading

In [69]:
lock = threading.Lock()

In [70]:
returnDict = {}

In [71]:
#for each task
def activity(activities,task,path):
    
    f.write(f"{datetime.now()};{path}.{task} Entry \n")
    print(task+"........")
    if(activities[task]['Type'] == 'Task'):
            #print(activities[task],activities[task].get('Condition'))
            if activities[task].get('Condition') is None:
                condition = ""
            else:
                condition = activities[task]['Condition']
                #print(condition)
        
            fun = eval(activities[task]['Function'])
            if activities[task]['Function'] == "DataLoad" :
                returnDict[path+'.'+task+'.NoOfDefects'] = fun(activities[task]['Inputs'],path+"."+task,"",condition)[1]
            else:
                fun(activities[task]['Inputs'],path+"."+task,"",condition)
            #fun(activities[task]['Inputs'],path+"."+task,"")
    elif activities[task]['Type'] == 'Flow':
        if activities[task]['Execution'] == 'Sequential':
                Sequential(activities[task],path+"."+task)
        elif activities[task]['Execution'] == 'Concurrent':
                Concurrent(activities[task],path+"."+task)
        f.write(f"{datetime.now()};{path}.{task} Exit \n")
    

In [72]:
def Concurrent(main_flow,path):
    activities = main_flow['Activities']
    
    thread= []
    
    for task in activities:
        #f.write(f"{datetime.now()};{path}.{task} Entry \n")
        t = threading.Thread(target=activity,args=(activities,task,path))
        t.start()
        thread.append(t)
    
    for t in thread:
        t.join()
        

In [73]:
def Sequential(main_flow,path):
    activities = main_flow['Activities']
    #print(main_flow)

    for task in activities:
        #print(task)
        f.write(f"{datetime.now()};{path}.{task} Entry \n")
        print(task," Entry .......")

        if activities[task]['Type'] == 'Task':
            print(activities[task],activities[task].get('Condition'))
            if activities[task].get('Condition') is None:
                condition = ""
            else:
                condition = activities[task]['Condition']
                print(condition)
            fun = eval(activities[task]['Function'])
            if activities[task]['Function'] == "DataLoad" :
                print(path+'.'+task+'.NoOfDefects')
                returnDict[path+'.'+task+'.NoOfDefects'] = fun(activities[task]['Inputs'],path+"."+task,"",condition)[1]
            else:
                fun(activities[task]['Inputs'],path+"."+task,"",condition)
        elif activities[task]['Type'] == 'Flow':
            if activities[task]['Execution'] == 'Sequential':
                Sequential(activities[task],path+"."+task)
            elif activities[task]['Execution'] == 'Concurrent':
                Concurrent(activities[task],path+"."+task)
            f.write(f"{datetime.now()};{path}.{task} Exit \n")
        
        

In [74]:
if main_flow['Execution'] == 'Sequential':
    Sequential(main_flow,"M2A_Workflow")
    f.write(f"{datetime.now()};M2A_Workflow Exit\n")

TaskA  Entry .......
{'Type': 'Task', 'Function': 'DataLoad', 'Inputs': {'Filename': 'Milestone2A_DataInput1.csv'}, 'Outputs': ['DataTable', 'NoOfDefects']} None
M2A_Workflow.TaskA.NoOfDefects

Milestone2A_DataInput1.csv
done.....
TaskB  Entry .......
{'Type': 'Task', 'Function': 'TimeFunction', 'Inputs': {'FunctionInput': 'TaskB_Input', 'ExecutionTime': '2'}} None

executing ......
done...
FlowA  Entry .......
TaskC........TaskD........
$(M2A_Workflow.TaskA.NoOfDefects) > 4
M2A_Workflow.TaskA.NoOfDefectsFlowB........


14
true
Milestone2A_DataInput2.csv
TaskE........
done.....
TaskF........

executing ......
done...
$(M2A_Workflow.FlowA.TaskC.NoOfDefects) < 6
M2A_Workflow.FlowA.TaskC.NoOfDefects
10
M2A_Workflow.FlowA.FlowB.TaskE skipped
false

executing ......
done...
